In [1]:
import sys,os
import xarray as xr
import numpy as np
import scipy as spy
import pickle 

import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from collections import OrderedDict 

import time as timestamp 
import importlib

sys.path.append("/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/")
import LIM_utils as lim

import LIM_utils_kb as limkb
import LIM_stats_kb as statskb
import LIM_plot_kb as plotkb
import LIM_building as limbuild

sys.path.append("/home/disk/kalman2/mkb22/pyLMR/")
import LMR_utils

import run_forecast_model_data as rf

from datetime import date

today = date.today()

#Year-month-day
today_date = today.strftime("%Y%m%d")

In [2]:
#--------------------------------------------------
# START USER PARAMETERS
#--------------------------------------------------
# number of EOFs to retain for the LIM state vector
ntrunc= 50
#nmodes = 15 # number of coupled EOFs for LIM state space (truncated-state space)
#nmodes = 20
nmodes_sic = 50
#modes_sic = 20

mo='all'
#mo=0

# forecast lead time in months that defines the LIM training
tau = 1

#limvars = ['sic']
limvars_nosic = []
#nvars= len(limvars)

# specify the model source 
#train_dsource = 'cmip6_cesm2_hist'
#train_dsource = 'cmip6_mpi_hist_ssp585'
#train_dsource ='satellite'
#train_dsource = 'mpi_hist_kb'
#train_dsource = 'ccsm4_lm_kb'
#train_dsource = 'satellite'
train_dsource = 'cesm_lme'
# train_dsource = 'era5'
# valid_dsource = 'era5'
valid_dsource = 'cesm_le_lmegrid'
#valid_dsource = 'satellite'
#valid_dsource = 'mpi_lm_kb'
#valid_dsource = 'mpi_lm_kb'
#valid_dsource = 'cmip6_cesm2_hist'

lat_cutoff_dict = {'tas':40,'psl':40,'zg':40,'tos':40,'sit':40,'sic':40}

sic_separate = True
Insamp = False

exp_setup = {}
exp_setup['limvars'] = ['tas','psl','zg','tos','sit','sic']
exp_setup['lat_cutoff'] = lat_cutoff_dict
exp_setup['Weight']=True
exp_setup['remove_climo'] = True
exp_setup['detrend'] = True
exp_setup['nyr_train'] = None

lags = [0,1,2,3,4,5,6,7,8]

yrend = 1650
#yrstart = 1056    # validation 1905-2005, 100 years of validation
#yrstart = 1081  # validation 1930-2005, 75 years of validation 
yrstart = 1  # validation 1915-2015, 75 years of validation 

# LME settings (out of sample): 
exp_setup['nyearsvalid'] = 75
exp_setup['nyearstrain'] = (yrend-850)
exp_setup['nyearstot'] = 1155
exp_setup['nyears_startvalid'] = yrstart*12

# #LME settings (in sample): 
# exp_setup['nyearsvalid'] = 100
# exp_setup['nyearstrain'] = (yrend-850)
# exp_setup['nyears_startvalid'] = 1*12
# exp_setup['nyearstot'] = 1155

# LM settings
# exp_setup['nyearsvalid'] = 10
# exp_setup['nyearstot'] = 1000
# exp_setup['nyears_startvalid'] = 900*12

# date_of_interest = '20210910'
today_date = '20211202'

master_save = False
save_decomp = False

#--------------------------------------------------
# END USER PARAMETERS
#--------------------------------------------------

if 'hist_ssp585' in train_dsource: 
    folder_add = 'hist_ssp585_concatenated/'
elif 'hist' in train_dsource: 
    folder_add = 'historical/'
elif 'lm' in train_dsource: 
    folder_add = 'last_millennium/'
elif 'satellite' in train_dsource: 
    folder_add = 'satellite/'
elif 'era5' in train_dsource: 
    folder_add = 'reanalysis/'
elif 'lme' in train_dsource: 
    folder_add = 'last_millennium/'

#mod_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/truncated_model_data/'+folder_add
mod_folder = '/home/disk/kalman2/mkb22/SI_LIMs/truncated_data/'+folder_add
#save_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/sensitivity_testing/neofs/'+folder_add
save_folder = '/home/disk/kalman2/mkb22/SI_LIMs/sensitivity_testing/nvalid_hist/cesm1/'

exp_setup['mo'] = mo
if 'all' in str(mo): 
    exp_setup['ind_month_trunc'] = False
else: 
    exp_setup['ind_month_trunc'] = True
exp_setup['tau'] = tau
exp_setup['train_dsource'] = train_dsource
exp_setup['valid_dsource'] = valid_dsource 
exp_setup['sic_separate'] = sic_separate
exp_setup['Insamp'] = Insamp
exp_setup['mod_folder'] = mod_folder
exp_setup['lags'] = lags
exp_setup['Insamp'] = Insamp
exp_setup['step2_trunc'] = False
exp_setup['ntrunc'] = ntrunc 
exp_setup['nmodes_sic'] = nmodes_sic

month_names = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec']

f_folder = save_folder
ar1f_folder = save_folder


#     mod_filename = ('_ntrunc'+str(ntrunc)+'_002_monthall_'+train_dsource+'_latcutoff_'+
#                      str(exp_setup['lat_cutoff'])+'_wtTrue_dtTrue_ntrain_850_'+str(yrend)+'_20211014.pkl')
#     mod_sic_filename = ('_ntrunc'+str(nmodes_sic_list[n])+'_002_monthall_'+train_dsource+'_latcutoff_'+
#                         str(exp_setup['lat_cutoff'])+'_wtTrue_dtTrue_ntrain_850_'+str(yrend)+'_20211014.pkl')

mod_filename_dict = {}
for var in exp_setup['limvars']:
    if 'sic' in var: 
        mod_filename_dict[var] = ('_ntrunc'+str(nmodes_sic)+'_002_monthall_'+train_dsource+'_latcutoff_'+
                                  str(exp_setup['lat_cutoff'][var])+'_wtTrue_dtTrue_ntrain_850_'+
                                  str(yrend)+'_20211014.pkl')
    else: 
        mod_filename_dict[var] = ('_ntrunc'+str(ntrunc)+'_002_monthall_'+train_dsource+'_latcutoff_'+
                                  str(exp_setup['lat_cutoff'][var])+'_wtTrue_dtTrue_ntrain_850_'+
                                  str(yrend)+'_20211014.pkl')

exp_setup['mod_filename'] = mod_filename_dict
#     exp_setup['mod_sic_filename'] = mod_sic_filename


In [3]:
#--------------------------------------------------
### Build L from truncated data: 
#--------------------------------------------------

LIMd = rf.build_L(exp_setup, save_folder, save=master_save)

if LIMd['npos_eigenvalues'] >0: 
    adj = True
else: 
    adj = False
exp_setup['adj'] = adj

Loading truncated tas from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/tas_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated psl from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/psl_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated zg from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/zg_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated tos from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/tos_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated sit from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/sit_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated sic 

In [4]:
#--------------------------------------------------
### Run Forecast: 
#--------------------------------------------------

forecast = rf.run_forecast(LIMd, exp_setup, f_folder, verbose=True, save=master_save, save_decomp=False)

Validation data: cesm_le_lmegrid
tecut = 76, tscut = 1.0
Loading from tas
Loading from ata/CESM_LE/tas_CESM_LE_002_regrid_cesm_lme_200601-210012.nc
time dimension: 2007 - 2082
detrending...
Working on month...0
Working on month...1
Working on month...2
Working on month...3
Working on month...4
Working on month...5
Working on month...6
Working on month...7
Working on month...8
Working on month...9
Working on month...10
Working on month...11
latitude cutoff detected: 40
-----------------------------------------------------
completed in 8.506826639175415 seconds
-----------------------------------------------------
Validation shape: (3888, 900)
time dimension: 2007 - 2082
(900,)
tecut = 76, tscut = 1.0
Loading from psl
Loading from ata/CESM_LE/psl_CESM_LE_002_regrid_cesm_lme_200601-210012.nc
time dimension: 2007 - 2082
detrending...
Working on month...0
Working on month...1
Working on month...2
Working on month...3
Working on month...4
Working on month...5
Working on month...6
Working on 

In [5]:
forecast_validation = rf.validate_forecast_monthly(forecast, exp_setup['limvars'], 1, exp_setup, LIMd, f_folder, 
                                                   iplot=False, save=master_save)

forecast_validation_lags = rf.validate_forecast_lagged(forecast, exp_setup['limvars'], exp_setup, LIMd, 
                                                       f_folder, iplot=False, save=master_save, 
                                                       detrend_truth=True)

Loading from cuments/SeaIceData/LME/LIMs/areacella_CESM_LME_global_001.nc
latitude cutoff detected: 40
Loading from cuments/SeaIceData/LME/LIMs/areacella_CESM_LME_global_001.nc
latitude cutoff detected: 40
Loading from cuments/SeaIceData/LME/LIMs/areacella_CESM_LME_global_001.nc
latitude cutoff detected: 40
Loading from b22/Documents/SeaIceData/LME/LIMs/areacello_global_LME_02.nc
latitude cutoff detected: 40
Loading from b22/Documents/SeaIceData/LME/LIMs/areacello_global_LME_02.nc
latitude cutoff detected: 40
Loading from b22/Documents/SeaIceData/LME/LIMs/areacello_global_LME_02.nc
latitude cutoff detected: 40
tecut = 76, tscut = 1.0
Loading from tas
Loading from ata/CESM_LE/tas_CESM_LE_002_regrid_cesm_lme_200601-210012.nc
time dimension: 2007 - 2082
detrending...
Working on month...0
Working on month...1
Working on month...2
Working on month...3
Working on month...4
Working on month...5
Working on month...6
Working on month...7
Working on month...8
Working on month...9
Working on mont

detrending...
Working on month...0
Found with nans (not in satellite), going to fill with zeros...
Working on month...1
Found with nans (not in satellite), going to fill with zeros...
Working on month...2
Found with nans (not in satellite), going to fill with zeros...
Working on month...3
Found with nans (not in satellite), going to fill with zeros...
Working on month...4
Found with nans (not in satellite), going to fill with zeros...
Working on month...5
Found with nans (not in satellite), going to fill with zeros...
Working on month...6
Found with nans (not in satellite), going to fill with zeros...
Working on month...7
Found with nans (not in satellite), going to fill with zeros...
Working on month...8
Found with nans (not in satellite), going to fill with zeros...
Working on month...9
Found with nans (not in satellite), going to fill with zeros...
Working on month...10
Found with nans (not in satellite), going to fill with zeros...
Working on month...11
Found with nans (not in sate

Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 2
Trained using month 2...
Validating against month 2
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 3
Trained using month 3...
Validating against month 3
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 4
Trained using month 4...
Validating against month 4
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 5
Trained using month 5...
Validating against month 5
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 6
Trained using month 6...
Validating against month 6
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 7
Trained using 

Working on month...1
Found with nans (not in satellite), going to fill with zeros...
Working on month...2
Found with nans (not in satellite), going to fill with zeros...
Working on month...3
Found with nans (not in satellite), going to fill with zeros...
Working on month...4
Found with nans (not in satellite), going to fill with zeros...
Working on month...5
Found with nans (not in satellite), going to fill with zeros...
Working on month...6
Found with nans (not in satellite), going to fill with zeros...
Working on month...7
Found with nans (not in satellite), going to fill with zeros...
Working on month...8
Found with nans (not in satellite), going to fill with zeros...
Working on month...9
Found with nans (not in satellite), going to fill with zeros...
Working on month...10
Found with nans (not in satellite), going to fill with zeros...
Working on month...11
Found with nans (not in satellite), going to fill with zeros...
latitude cutoff detected: 40
----------------------------------

In [6]:
#--------------------------------------------------
### Run AR1 Forecast: 
#--------------------------------------------------

valid_vars=exp_setup['limvars']

ar1cast = rf.ar1_forecast_valid_by_month(LIMd['P_train'], forecast['P_train_valid'], LIMd,
                                         exp_setup, valid_vars, month_names, ar1f_folder, forecast,
                                         lag=None, iplot=False, save=master_save, save_decomp=False)

ar1cast_lags = rf.ar1_forecast_valid_by_lag(LIMd['P_train'], forecast['P_train_valid'], LIMd, exp_setup, 
                                            exp_setup['limvars'], month_names, ar1f_folder, forecast,
                                            iplot=False, save=master_save, save_decomp=False,
                                            detrend_truth=False)

Loading from cuments/SeaIceData/LME/LIMs/areacella_CESM_LME_global_001.nc
latitude cutoff detected: 40
Loading from cuments/SeaIceData/LME/LIMs/areacella_CESM_LME_global_001.nc
latitude cutoff detected: 40
Loading from cuments/SeaIceData/LME/LIMs/areacella_CESM_LME_global_001.nc
latitude cutoff detected: 40
Loading from b22/Documents/SeaIceData/LME/LIMs/areacello_global_LME_02.nc
latitude cutoff detected: 40
Loading from b22/Documents/SeaIceData/LME/LIMs/areacello_global_LME_02.nc
latitude cutoff detected: 40
Loading from b22/Documents/SeaIceData/LME/LIMs/areacello_global_LME_02.nc
latitude cutoff detected: 40
Lead = 1
tecut = 76, tscut = 1.0
Loading from tas
Loading from ata/CESM_LE/tas_CESM_LE_002_regrid_cesm_lme_200601-210012.nc
time dimension: 2007 - 2082
detrending...
Working on month...0
Working on month...1
Working on month...2
Working on month...3
Working on month...4
Working on month...5
Working on month...6
Working on month...7
Working on month...8
Working on month...9
Workin

detrending...
Working on month...0
Found with nans (not in satellite), going to fill with zeros...
Working on month...1
Found with nans (not in satellite), going to fill with zeros...
Working on month...2
Found with nans (not in satellite), going to fill with zeros...
Working on month...3
Found with nans (not in satellite), going to fill with zeros...
Working on month...4
Found with nans (not in satellite), going to fill with zeros...
Working on month...5
Found with nans (not in satellite), going to fill with zeros...
Working on month...6
Found with nans (not in satellite), going to fill with zeros...
Working on month...7
Found with nans (not in satellite), going to fill with zeros...
Working on month...8
Found with nans (not in satellite), going to fill with zeros...
Working on month...9
Found with nans (not in satellite), going to fill with zeros...
Working on month...10
Found with nans (not in satellite), going to fill with zeros...
Working on month...11
Found with nans (not in sate

Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 2
Trained using month 2...
Validating against month 2
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 3
Trained using month 3...
Validating against month 3
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 4
Trained using month 4...
Validating against month 4
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 5
Trained using month 5...
Validating against month 5
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 6
Trained using month 6...
Validating against month 6
Truth_anom shape: (30720, 75)
Forecast_anom shape: (30720, 75)
changing cellarea units from centimeter^2 to km^2
Month 7
Trained using 

Working on month...5
Found with nans (not in satellite), going to fill with zeros...
Working on month...6
Found with nans (not in satellite), going to fill with zeros...
Working on month...7
Found with nans (not in satellite), going to fill with zeros...
Working on month...8
Found with nans (not in satellite), going to fill with zeros...
Working on month...9
Found with nans (not in satellite), going to fill with zeros...
Working on month...10
Found with nans (not in satellite), going to fill with zeros...
Working on month...11
Found with nans (not in satellite), going to fill with zeros...
latitude cutoff detected: 40
-----------------------------------------------------
completed in 89.97273111343384 seconds
-----------------------------------------------------
Lag 1
Trained using all months...
Truth_anom shape: (30720, 899)
Forecast_anom shape: (30720, 899)
changing cellarea units from centimeter^2 to km^2
Lag 2
Trained using all months...
Truth_anom shape: (30720, 898)
Forecast_anom

In [7]:
LIMcast = {}

LIMcast['LIMd'] = LIMd
LIMcast['forecast'] = forecast
LIMcast['forecast_validation'] = forecast_validation
LIMcast['forecast_validation_lags'] = forecast_validation_lags
LIMcast['ar1cast'] = ar1cast
LIMcast['ar1cast_lags'] = ar1cast_lags

if save_decomp is False: 
    if 'x_forecast_dcomp' in LIMcast['forecast'].keys():
        LIMcast['forecast'].pop('x_forecast_dcomp')

start_yr = str(forecast['var_dict_valid']['sic']['time'][0])[0:4]
end_yr = str(forecast['var_dict_valid']['sic']['time'][-1])[0:4]
varsname = '_'.join([f'{x}{ntrunc}L{exp_setup["lat_cutoff"][x]}' for x in exp_setup['limvars']])

filename_end = (exp_setup['train_dsource']+'_002_ntrain'+exp_setup['mod_filename']['tas'][-22:-13]+
                '_validyrs_'+start_yr+'_'+end_yr+'_'+varsname+'_'+today_date+'.pkl')

# filename_end = (exp_setup['train_dsource']+'_002_ntrain'+exp_setup['mod_filename'][-22:-13]+
#                 '_validyrs_'+start_yr+'_'+end_yr+'_'+
#                 (str(exp_setup['ntrunc'])+"_").join(exp_setup['limvars'])+
#                 str(exp_setup['nmodes_sic'])+'_'+today_date+'.pkl')

print('saving in: '+save_folder+'LIMcast_'+filename_end)
pickle.dump(LIMcast, open(save_folder+'LIMcast_'+filename_end, "wb" ) )

saving in: /home/disk/kalman2/mkb22/SI_LIMs/sensitivity_testing/nvalid_hist/cesm1/LIMcast_cesm_lme_002_ntrain_850_1650_validyrs_2007_2082_tas50L40_psl50L40_zg50L40_tos50L40_sit50L40_sic50L40_20211202.pkl


In [10]:
fdic_valid = limkb.build_training_dic('cesm_le')

Ptrunc_valid = {}
var_dict_valid = {}
ntims = len(exp_setup['lags'])
print('Validation data: '+exp_setup['valid_dsource'])

Validation data: cesm_le


In [11]:
var = 'tas'
verbose=True

tecut = int(exp_setup['nyears_startvalid']/12)+(exp_setup['nyearsvalid'])
#        tecut = exp_setup['nyearstot'] - (int(exp_setup['nyears_startvalid']/12)+(exp_setup['nyearsvalid']))

print('tecut = '+str(tecut) +', tscut = '+str(exp_setup['nyears_startvalid']/12))

X_var_valid, var_dict_valid = limkb.load_data(var, var_dict_valid, fdic_valid, 
                                              remove_climo=exp_setup['remove_climo'], 
                                              detrend=exp_setup['detrend'], verbose=verbose, 
                                              tscut=int(exp_setup['nyears_startvalid']/12), 
                                              tecut=tecut, lat_cutoff=exp_setup['lat_cutoff'][var])
if 'sic' in var:
    if np.nanmax(X_var_valid)>1:
        print('Changing units of sic be a between 0 to 1')
        X_var_valid = X_var_valid/100

print('Validation shape: '+str(X_var_valid.shape))

if 'datetime64' in str(type(var_dict_valid[var]['time'][0])):
    print('time dimension: '+str(var_dict_valid[var]['time'][0].astype('M8[Y]'))+' - '+
          str(var_dict_valid[var]['time'][-1].astype('M8[Y]')))
    print(var_dict_valid[var]['time'].shape)
else: 
    print('time dimension: '+str(var_dict_valid[var]['time'][0].year)+' - '+
          str(var_dict_valid[var]['time'][-1].year))
    print(var_dict_valid[var]['time'].shape)

tecut = 76, tscut = 1.0
Loading from tas
Loading from ocuments/SeaIceData/CESM_LE/tas_CESM_LE_002_200601-210012.nc
time dimension: 2007 - 2082
detrending...
Working on month...0
Working on month...1
Working on month...2
Working on month...3
Working on month...4
Working on month...5
Working on month...6
Working on month...7
Working on month...8
Working on month...9
Working on month...10
Working on month...11
latitude cutoff detected: 40
-----------------------------------------------------
completed in 39.741795778274536 seconds
-----------------------------------------------------
Validation shape: (15552, 900)
time dimension: 2007 - 2082
(900,)


In [12]:
Ptrunc_valid[var] = limkb.step1_projection_validation_var(X_var_valid, LIMd['E3'][var], 
                                                          LIMd['standard_factor'][var],
                                                          LIMd['W_all'][var], Weights=exp_setup['Weight'])

ValueError: operands could not be broadcast together with shapes (3888,1) (15552,900) 

In [14]:
LIMd['E3'][var].shape, LIMd['W_all'][var].shape, X_var_valid.shape

((3888, 50), (3888,), (15552, 900))

In [26]:
startyr = 801
yrtot = 1001
yrvalid = 200

In [34]:
if (yrtot-startyr)/yrvalid >1:
    nloop = int(np.floor((yrtot-startyr)/yrvalid))
else: 
    nloop = 1

In [35]:
for i in range(nloop):
    startyr = startyr+i*yrvalid
    print(startyr)

1001


In [33]:
nloop, (yrtot-startyr)/yrvalid

(0, 0.0)

In [38]:
%run run_forecast_LME_nvalid_sliding_sensitivity_120221.py

Loading truncated tas from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/tas_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated psl from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/psl_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated zg from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/zg_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated tos from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/tos_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated sit from: /home/disk/kalman2/mkb22/SI_LIMs/truncated_data/last_millennium/sit_ntrunc50_002_monthall_cesm_lme_latcutoff_40_wtTrue_dtTrue_ntrain_850_1650_20211014.pkl
Month is all
Loading truncated sic 

Loading from cuments/SeaIceData/LME/LIMs/areacella_CESM_LME_global_001.nc
latitude cutoff detected: 40
Loading from cuments/SeaIceData/LME/LIMs/areacella_CESM_LME_global_001.nc
latitude cutoff detected: 40
Loading from cuments/SeaIceData/LME/LIMs/areacella_CESM_LME_global_001.nc
latitude cutoff detected: 40
Loading from b22/Documents/SeaIceData/LME/LIMs/areacello_global_LME_02.nc
latitude cutoff detected: 40
Loading from b22/Documents/SeaIceData/LME/LIMs/areacello_global_LME_02.nc
latitude cutoff detected: 40
Loading from b22/Documents/SeaIceData/LME/LIMs/areacello_global_LME_02.nc
latitude cutoff detected: 40
tecut = 1001, tscut = 801.0
Loading from tas
Loading from a/LME/LIMs/tas_sfc_Amon_CESM_LMEallforc_002_085001-200512.nc
time dimension: 1651 - 1850
detrending...
Working on month...0
Working on month...1
Working on month...2
Working on month...3
Working on month...4
Working on month...5
Working on month...6
Working on month...7
Working on month...8
Working on month...9
Working on 

time dimension: 1651 - 1850
detrending...
Working on month...0
Found with nans (not in satellite), going to fill with zeros...
Working on month...1
Found with nans (not in satellite), going to fill with zeros...
Working on month...2
Found with nans (not in satellite), going to fill with zeros...
Working on month...3
Found with nans (not in satellite), going to fill with zeros...
Working on month...4
Found with nans (not in satellite), going to fill with zeros...
Working on month...5
Found with nans (not in satellite), going to fill with zeros...
Working on month...6
Found with nans (not in satellite), going to fill with zeros...
Working on month...7
Found with nans (not in satellite), going to fill with zeros...
Working on month...8
Found with nans (not in satellite), going to fill with zeros...
Working on month...9
Found with nans (not in satellite), going to fill with zeros...
Working on month...10
Found with nans (not in satellite), going to fill with zeros...
Working on month...11


Truth_anom shape: (30720, 199)
Forecast_anom shape: (30720, 199)
changing cellarea units from centimeter^2 to km^2
Month 1
Trained using month 1...
Validating against month 1
Truth_anom shape: (30720, 200)
Forecast_anom shape: (30720, 200)
changing cellarea units from centimeter^2 to km^2
Month 2
Trained using month 2...
Validating against month 2
Truth_anom shape: (30720, 200)
Forecast_anom shape: (30720, 200)
changing cellarea units from centimeter^2 to km^2
Month 3
Trained using month 3...
Validating against month 3
Truth_anom shape: (30720, 200)
Forecast_anom shape: (30720, 200)
changing cellarea units from centimeter^2 to km^2
Month 4
Trained using month 4...
Validating against month 4
Truth_anom shape: (30720, 200)
Forecast_anom shape: (30720, 200)
changing cellarea units from centimeter^2 to km^2
Month 5
Trained using month 5...
Validating against month 5
Truth_anom shape: (30720, 200)
Forecast_anom shape: (30720, 200)
changing cellarea units from centimeter^2 to km^2
Month 6
Tr

tecut = 1001, tscut = 801.0
Loading from tos
Loading from a/LME/LIMs/tos_sfc_Omon_CESM_LMEallforc_002_085001-200512.nc
time dimension: 1651 - 1850
detrending...
Working on month...0
Found with nans (not in satellite), going to fill with zeros...
Working on month...1
Found with nans (not in satellite), going to fill with zeros...
Working on month...2
Found with nans (not in satellite), going to fill with zeros...
Working on month...3
Found with nans (not in satellite), going to fill with zeros...
Working on month...4
Found with nans (not in satellite), going to fill with zeros...
Working on month...5
Found with nans (not in satellite), going to fill with zeros...
Working on month...6
Found with nans (not in satellite), going to fill with zeros...
Working on month...7
Found with nans (not in satellite), going to fill with zeros...
Working on month...8
Found with nans (not in satellite), going to fill with zeros...


KeyboardInterrupt: 